In [36]:
from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pickle
import requests
import json
import re
from tqdm import tqdm

In [37]:
directory = '/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/'

In [38]:
subway = pd.read_csv(directory + 'data/CARD_SUBWAY_MONTH_202110.csv')

In [39]:
subway = subway.reset_index()
subway.columns = ['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자','na']
subway = subway.drop(columns = 'na')

In [40]:
subway_nm = subway.역명.unique()

In [41]:
st = pd.DataFrame(columns=['subway_name','위도','경도'])
st['subway_name'] = subway_nm
val = pd.DataFrame(st['subway_name'].value_counts()).reset_index().rename(columns={'index':'subway_name','subway_name':'환승역 수'})
st = st.drop_duplicates().reset_index(drop=True)
st = pd.merge(st,val,how='inner',on='subway_name')

In [42]:
st['subway_name'] = st['subway_name'].apply(lambda x : re.sub(r'\([^)]*\)','',x)) #괄호 포함 글자 제거

In [47]:
st['subway_name'] = list(map(lambda x: x+'역', st['subway_name']))

In [48]:
def get_location():
    
    subway = np.array(st['subway_name'])
    
    for i in tqdm(subway):
        
        url = "https://dapi.kakao.com/v2/local/search/keyword.json"

        queryString = {'query' : i}
        header = {'Authorization': 'KakaoAK 2faece6ee426f2b12f9f597dd2f0f9d0'} #rest api

        response = requests.get(url, headers=header, params=queryString)
        tokens = response.json()
        
        try: 
            lat = tokens['documents'][0]['x']
            lng = tokens['documents'][0]['y']
            st.loc[st['subway_name'].isin([i]),'위도'] = lat
            st.loc[st['subway_name'].isin([i]),'경도'] = lng
        except:
            print(i)
            pass

In [49]:
get_location()

100%|█████████████████████████████████████████| 521/521 [02:27<00:00,  3.53it/s]


In [50]:
subway_location = st[['subway_name','위도','경도']]

In [52]:
subway_location[subway_location.subway_name == '압구정역']

,subway_name,위도,경도
222,압구정역,127.02850865895756,37.52649127416921


In [53]:
subway_location.to_csv(directory + 'data/subway_location.csv', index = False)

In [31]:
url = "https://dapi.kakao.com/v2/local/search/keyword.json"

queryString = {'query' : '압구정역'}
header = {'Authorization': 'KakaoAK 2faece6ee426f2b12f9f597dd2f0f9d0'} #rest api

response = requests.get(url, headers=header, params=queryString)
tokens = response.json()

In [32]:
tokens

{'documents': [{'address_name': '서울 강남구 신사동 668',
   'category_group_code': 'SW8',
   'category_group_name': '지하철역',
   'category_name': '교통,수송 > 지하철,전철 > 수도권3호선',
   'distance': '',
   'id': '21160592',
   'phone': '02-6110-3361',
   'place_name': '압구정역 3호선',
   'place_url': 'http://place.map.kakao.com/21160592',
   'road_address_name': '서울 강남구 압구정로 지하 172',
   'x': '127.02850865895756',
   'y': '37.52649127416921'},
  {'address_name': '서울 강남구 압구정동 495',
   'category_group_code': 'SW8',
   'category_group_name': '지하철역',
   'category_name': '교통,수송 > 지하철,전철 > 수인분당선',
   'distance': '',
   'id': '18234115',
   'phone': '1544-7788',
   'place_name': '압구정로데오역 수인분당선',
   'place_url': 'http://place.map.kakao.com/18234115',
   'road_address_name': '서울 강남구 압구정로 지하 402',
   'x': '127.0406027693898',
   'y': '37.5275184818021'},
  {'address_name': '서울 강남구 신사동 611',
   'category_group_code': '',
   'category_group_name': '',
   'category_name': '가정,생활 > 생활용품점 > 다이소',
   'distance': '',
   'id': '

In [30]:
tokens['documents'][0]['x']

'127.029112581665'